## Interactive wind data testing
Interactively test data from data.csv. Use to both test wind algorithms and determine what data is needed.

In [1]:
#"magic" uses ipympl
%matplotlib widget  
import pandas as pd
import matplotlib.pyplot as plt
import envirodataqc as envqc

### Load data.csv

In [2]:
data = pd.read_csv('../data/data.csv', index_col='timestamp', parse_dates=True)

In [3]:
data.head()

,temperature_C,humidity_%,pressure_mb,windspeed_10min_avg,winddir_10min_avg
timestamp,,,,,
2021-03-07 00:00:00,-2.2,59.7,811.5,1.2,179.1
2021-03-07 00:10:00,-4.0,64.9,811.6,2.1,288.1
2021-03-07 00:20:00,-4.4,66.0,811.7,2.4,317.2
2021-03-07 00:30:00,-4.0,65.3,811.6,2.3,319.2
2021-03-07 00:40:00,-4.1,65.0,811.6,2.0,320.2


## Extract wind data and check vals

In [4]:
#Calculate flags on wind data
windsp = envqc.check_vals(data[['windspeed_10min_avg']].copy(),'wind_speed')
winddir = envqc.check_vals(data[['winddir_10min_avg']].copy(),'wind_direction')


In [5]:
windsp['maxflag'] = windsp[['flags_range','flags_rate','flags_flat']].max(1)
winddir['maxflag'] = winddir[['flags_range','flags_rate','flags_flat']].max(1)

In [6]:
windsp_susp = windsp.loc[windsp.maxflag == 1,'windspeed_10min_avg']
windsp_bad = windsp.loc[windsp.maxflag ==2,'windspeed_10min_avg']
winddir_susp = winddir.loc[winddir.maxflag == 1,'winddir_10min_avg']
winddir_bad = winddir.loc[winddir.maxflag == 2,'winddir_10min_avg']

In [23]:
fig, axs = plt.subplots(2,1,sharex=True)
windsp['windspeed_10min_avg'].plot(ax = axs[0])
winddir['winddir_10min_avg'].plot(ax = axs[1])
wind_susp = windsp.loc[windsp.maxflag > 0,'windspeed_10min_avg']
wind_bad = windsp.loc[windsp.maxflag > 0,'windspeed_10min_avg']

if(not windsp_susp.empty): windsp_susp.plot(ax=axs[0],style='*r')
if(not windsp_bad.empty): windsp_bad.plot(ax=axs[0],style='*k')
if(not winddir_susp.empty): winddir_susp.plot(ax=axs[1],style='*r')
if(not winddir_bad.empty): winddir_bad.plot(ax=axs[1],style='*k')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### General
* Check vals is correctly identifying suspicious or bad flatlining in input data. The section of unflagged wind direction data near 11:00 3/7 is actually just changing by very small amounts.
#### Focus on 3/8 around 6:00
* There are two zero wind sections here. Both are not flagged because 0 wind is ignored.
* There is a very short section of flatlined direction data starting at 9:00. It isn't flagged because it is too short.

## Assess wind specific algorithms

In [8]:
from envirodataqc import wind

In [9]:
windsp['flags_special'] = wind.check_windsp_withdir(
    data['windspeed_10min_avg'].to_numpy(),
    data['winddir_10min_avg'].to_numpy()
)

In [12]:
winddir['flags_special'] = wind.check_winddir_withsp(
    data['windspeed_10min_avg'].to_numpy(),
    data['winddir_10min_avg'].to_numpy()
)

In [17]:
windsp_susp2 = data.loc[windsp.flags_special == 1,'windspeed_10min_avg']
winddir_susp2 = data.loc[winddir.flags_special == 1,'winddir_10min_avg']

In [22]:
fig2, axs2 = plt.subplots(2,1,sharex=True)
data['windspeed_10min_avg'].plot(ax = axs2[0])
data['winddir_10min_avg'].plot(ax = axs2[1])

if(not windsp_susp2.empty): windsp_susp2.plot(ax=axs2[0],style='*r')
if(not winddir_susp2.empty): winddir_susp2.plot(ax=axs2[1],style='*r')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

* Small sections of direction data flatlining (near 00:00 3/7 and 10:00 3/8) are correctly flagged because wind speed is above zero.
* A section of zero wind near 6:00 3/8 is flagged because direction data is changing.
* Another near by section of zero wind is not flagged because direction data is also flat.